# Installation

In [1]:
!git lfs install
!git clone https://github.com/SalomePx/NLP-Essay-Scoring

Error: Failed to call git rev-parse --git-dir --show-toplevel: "fatal: not a git repository (or any of the parent directories): .git\n"
Git LFS initialized.
Cloning into 'NLP-Essay-Scoring'...
remote: Enumerating objects: 79, done.
remote: Total 79 (delta 0), reused 0 (delta 0), pack-reused 79
Unpacking objects: 100% (79/79), done.
Checking out files: 100% (46/46), done.
Filtering content: 100% (5/5), 608.32 MiB | 78.71 MiB/s, done.


In [2]:
%cd NLP-Essay-Scoring

/content/NLP-Essay-Scoring


In [3]:
!pip install -r requirements.txt

     |████████████████████████████████| 647 kB 8.2 MB/s 
     |████████████████████████████████| 1.7 MB 41.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 125 kB 47.5 MB/s 
     |████████████████████████████████| 1.4 MB 35.5 MB/s 
     |████████████████████████████████| 256 kB 49.6 MB/s 
     |████████████████████████████████| 280 kB 61.2 MB/s 
     |████████████████████████████████| 136 kB 59.7 MB/s 
     |████████████████████████████████| 109 kB 49.4 MB/s 
     |████████████████████████████████| 843 kB 48.1 MB/s 
     |████████████████████████████████| 96 kB 5.8 MB/s 
     |████████████████████████████████| 546 kB 61.6 MB/s 
  Created wheel for swifter: filename=swifter-1.1.3-py3-none-any.whl size=13078 sha256=c054ac178a12c1c9aca91fc85c4838c866ee41103946970f6f62c91a5bf5ebd9
  Stored in directory: /root/.cache/pip/whee

# Topic Modeling Plots

In [ ]:
from topic_modeling import create_plots, spacy_analysis, prepare
import pickle5 as pickle5

training_set = spacy_analysis()
lda_base, doc_term_matrix, vectorizer = create_plots(training_set)

In [ ]:
from pyLDAvis.sklearn import prepare
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis

pyLDAvis.enable_notebook()
prepare(lda_base, doc_term_matrix, vectorizer, n_jobs=1)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.063518 -0.021168       1        1  21.754269
3      0.062968 -0.074080       2        1  21.325591
0      0.115176 -0.063530       3        1  15.188533
2     -0.429028 -0.020653       4        1  10.148675
4     -0.008827 -0.147362       5        1   9.729296
5      0.123018 -0.126696       6        1   9.215630
6      0.057578  0.278160       7        1   7.217582
1      0.015598  0.175331       8        1   5.420424, topic_info=           Term          Freq         Total Category  logprob  loglift
363    computer  20341.000000  20341.000000  Default  30.0000  30.0000
203        book  13905.000000  13905.000000  Default  29.0000  29.0000
230    building   7822.000000   7822.000000  Default  28.0000  28.0000
522   dirigible   6253.000000   6253.000000  Default  27.0000  27.0000
1162       mood   4312.000000   4312.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
1710      start    440.334937   3691.756131   Topic8  -4.6619   0.7887
1448     reason    369.176035   4302.204551   Topic8  -4.8382   0.4594
302      change    307.533573   1804.831577   Topic8  -5.0209   1.1454
413     country    278.055251   1467.547595   Topic8  -5.1216   1.2515
1122       mean    259.572748   2433.489559   Topic8  -5.1904   0.6770

[510 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
3         3  0.008427      abandon
3         5  0.990194      abandon
4         1  0.990129        abhor
13        4  0.994927  accommodate
18        5  0.989417         acid
...     ...       ...          ...
1996      3  0.128378        young
1996      5  0.022297        young
1996      6  0.027027        young
1996      7  0.047973        young
1996      8  0.012162        young

[1298 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 4, 1, 3, 5, 6, 7, 2])

# Demo

In [4]:
%cd NLP-Essay-Scoring

[Errno 2] No such file or directory: 'NLP-Essay-Scoring'
/content/NLP-Essay-Scoring


In [6]:
from ml_framework import get_features, get_df, get_predictors, get_X_y
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
"""training_set = get_features()
df = get_df(training_set)
predictors = get_predictors()
X, y = get_X_y(training_set, predictors)"""


reference_essays = {
    1: 161,
    2: 3022,
    3: 5263,
    4: 5341,
    5: 7209,
    6: 8896,
    7: 11796,
    8: 12340,
}  # topic: essay_id

references = {}


nlp = spacy.load("en_core_web_sm")
stop_words = set(STOP_WORDS)

# generate nlp object for reference essays:
for topic, index in reference_essays.items():
    references[topic] = nlp(training_set.iloc[index]["essay"])

  
similarity = training_set.apply(
          lambda row: nlp(row["essay"]).similarity(references[row["topic"]]), axis=1
      )

In [8]:
import language_tool_python
import swifter
import pandas as pd
import numpy as np
import pickle5 as pickle
from topic_modeling import STOP_WORDS, punctuation
from ml_framework import get_predictors
import spacy

with open("data/training_spacy.pkl", "rb") as fh:
    training_set = pickle.load(fh)

def preprocess_essay(essay_to_predict):
    d = {"essay" : [essay_to_predict]}
    essay_to_predict = pd.DataFrame(d)
    tool = language_tool_python.LanguageTool("en-US")

    essay_to_predict["matches"] = essay_to_predict["essay"].apply(lambda txt: tool.check(txt))
    essay_to_predict["corrections"] = essay_to_predict.apply(
        lambda l: len(l["matches"]), axis=1
    )
    essay_to_predict["corrected"] = essay_to_predict.swifter.apply(
        lambda l: tool.correct(l["essay"]), axis=1
    )
        
    essay_to_predict["word_count"] = essay_to_predict["essay"].str.strip().str.split().str.len()

    sents = []
    tokens = []
    lemma = []
    pos = []
    ner = []

    stop_words = set(STOP_WORDS)
    stop_words.update(punctuation)  # remove it if you need punctuation

    nlp = spacy.load("en_core_web_sm")

    # suppress numpy warnings
    np.warnings.filterwarnings("ignore")

    for essay in nlp.pipe(essay_to_predict["corrected"], batch_size=100):
        if essay.is_parsed:
            tokens.append([e.text for e in essay])
            sents.append([sent.text.strip() for sent in essay.sents])
            pos.append([e.pos_ for e in essay])
            ner.append([e.text for e in essay.ents])
            lemma.append([n.lemma_ for n in essay])
        else:
            # We want to make sure that the lists of parsed results have the
            # same number of entries of the original Dataframe, so add some blanks in case the parse fails
            tokens.append(None)
            lemma.append(None)
            pos.append(None)
            sents.append(None)
            ner.append(None)


    # generate document similarity for each essay compared to topic reference
    essay_to_predict["similarity"] = similarity
    essay_to_predict["tokens"] = tokens
    essay_to_predict["lemma"] = lemma
    essay_to_predict["pos"] = pos
    essay_to_predict["sents"] = sents
    essay_to_predict["ner"] = ner

    essay_to_predict["token_count"] = essay_to_predict.apply(lambda x: len(x["tokens"]), axis=1)
    essay_to_predict["unique_token_count"] = essay_to_predict.apply(
        lambda x: len(set(x["tokens"])), axis=1
    )
    essay_to_predict["nostop_count"] = essay_to_predict.apply(
        lambda x: len([token for token in x["tokens"] if token not in stop_words]),
        axis=1,
    )
    essay_to_predict["sent_count"] = essay_to_predict.apply(lambda x: len(x["sents"]), axis=1)
    essay_to_predict["ner_count"] = essay_to_predict.apply(lambda x: len(x["ner"]), axis=1)
    essay_to_predict["comma"] = essay_to_predict.apply(
        lambda x: x["corrected"].count(","), axis=1
    )
    essay_to_predict["question"] = essay_to_predict.apply(
        lambda x: x["corrected"].count("?"), axis=1
    )
    essay_to_predict["exclamation"] = essay_to_predict.apply(
        lambda x: x["corrected"].count("!"), axis=1
    )
    essay_to_predict["quotation"] = essay_to_predict.apply(
        lambda x: x["corrected"].count('"') + x["corrected"].count("'"), axis=1
    )
    essay_to_predict["organization"] = essay_to_predict.apply(
        lambda x: x["corrected"].count(r"@ORGANIZATION"), axis=1
    )
    essay_to_predict["caps"] = essay_to_predict.apply(
        lambda x: x["corrected"].count(r"@CAPS"), axis=1
    )
    essay_to_predict["person"] = essay_to_predict.apply(
        lambda x: x["corrected"].count(r"@PERSON"), axis=1
    )
    essay_to_predict["location"] = essay_to_predict.apply(
        lambda x: x["corrected"].count(r"@LOCATION"), axis=1
    )
    essay_to_predict["money"] = essay_to_predict.apply(
        lambda x: x["corrected"].count(r"@MONEY"), axis=1
    )
    essay_to_predict["time"] = essay_to_predict.apply(
        lambda x: x["corrected"].count(r"@TIME"), axis=1
    )
    essay_to_predict["date"] = essay_to_predict.apply(
        lambda x: x["corrected"].count(r"@DATE"), axis=1
    )
    essay_to_predict["percent"] = essay_to_predict.apply(
        lambda x: x["corrected"].count(r"@PERCENT"), axis=1
    )
    essay_to_predict["noun"] = essay_to_predict.apply(lambda x: x["pos"].count("NOUN"), axis=1)
    essay_to_predict["adj"] = essay_to_predict.apply(lambda x: x["pos"].count("ADJ"), axis=1)
    essay_to_predict["pron"] = essay_to_predict.apply(lambda x: x["pos"].count("PRON"), axis=1)
    essay_to_predict["verb"] = essay_to_predict.apply(lambda x: x["pos"].count("VERB"), axis=1)
    essay_to_predict["noun"] = essay_to_predict.apply(lambda x: x["pos"].count("NOUN"), axis=1)
    essay_to_predict["cconj"] = essay_to_predict.apply(
        lambda x: x["pos"].count("CCONJ"), axis=1
    )
    essay_to_predict["adv"] = essay_to_predict.apply(lambda x: x["pos"].count("ADV"), axis=1)
    essay_to_predict["det"] = essay_to_predict.apply(lambda x: x["pos"].count("DET"), axis=1)
    essay_to_predict["propn"] = essay_to_predict.apply(
        lambda x: x["pos"].count("PROPN"), axis=1
    )
    essay_to_predict["num"] = essay_to_predict.apply(lambda x: x["pos"].count("NUM"), axis=1)
    essay_to_predict["part"] = essay_to_predict.apply(lambda x: x["pos"].count("PART"), axis=1)
    essay_to_predict["intj"] = essay_to_predict.apply(lambda x: x["pos"].count("INTJ"), axis=1)

    predictors = get_predictors()
    return essay_to_predict[predictors]


In [9]:
from sklearn.ensemble import GradientBoostingClassifier

training_set = get_features()
X, y = get_X_y(training_set, predictors)
model = GradientBoostingClassifier(random_state=26)
model.fit(X, y)

GradientBoostingClassifier(random_state=26)

In [21]:
model.fit(X[:-1], y[:-1])

GradientBoostingClassifier(random_state=26)

In [10]:
!pip install gradio

     |████████████████████████████████| 2.9 MB 5.1 MB/s 
     |████████████████████████████████| 2.0 MB 54.8 MB/s 
     |████████████████████████████████| 84 kB 3.0 MB/s 
     |████████████████████████████████| 54 kB 2.5 MB/s 
     |████████████████████████████████| 212 kB 59.1 MB/s 
     |████████████████████████████████| 253 kB 49.6 MB/s 
     |████████████████████████████████| 1.1 MB 48.4 MB/s 
     |████████████████████████████████| 53 kB 1.9 MB/s 
     |████████████████████████████████| 94 kB 2.8 MB/s 
     |████████████████████████████████| 144 kB 67.0 MB/s 
     |████████████████████████████████| 271 kB 52.3 MB/s 
     |████████████████████████████████| 10.9 MB 39.6 MB/s 
     |████████████████████████████████| 58 kB 5.5 MB/s 
     |████████████████████████████████| 79 kB 8.1 MB/s 
     |████████████████████████████████| 43 kB 2.0 MB/s 
     |████████████████████████████████| 4.0 MB 41.2 MB/s 
     |████████████████████████████████| 61 kB 448 kB/s 
     |████████████████████████

In [24]:
from sklearn.linear_model import ElasticNet 
from sklearn.ensemble import ExtraTreesClassifier, RandomForestRegressor

def gradio_predict(essay):
    essay = preprocess_essay(essay)
    pred = model.predict(essay)
    print(f"Your grade is {pred}")

    return pred[0]

#essay = gradio_predict("""The afternoon grew so glowering that in the sixth inning the arc lights were turned on--always a wan sight in the daytime, like the burning headlights of a funeral procession. Aided by the gloom, Fisher was slicing through the Sox rookies, and Williams did not come to bat in the seventh. He was second up in the eighth. This was almost certainly his last time to come to the plate in Fenway Park, and instead of merely cheering, as we had at his three previous appearances, we stood, all of us, and applauded.""")

In [14]:
import gradio

## Example of essay from the valid set that get a 10 and an 8

In [26]:
#create interface object
gui = gradio.Interface(fn=gradio_predict, #callable function
                   inputs="text", #input format
                   outputs="text",
                   examples=["Many people believe that laughter can improve your life. Laughing lets out endorphins that give your body energy for daily activities. Relationships are usually better if both the people are happy. Happiness is like shaking up a soda bottle. Happy emotions build up in your body and when shaken, release with an explosion of positive emotions such as laughing. When I was born, my mother said that I was the most chubby baby in the world. As she said this, she laughed, in constant thought that I will eventually become a young adult. I giggled in the backseat of the car as we drove home together. I was taught by my mother, who laughed all the time until her eyes were full of tears, to laugh at every chance I had. Through my years as an infant, I giggled, chuckled, and laughed my way to adolescence. As I moved into adolescence, I was diagnosed with @ORGANIZATION1's @CAPS1, a mild autistic @CAPS1 that has a variation of side effects. Some people who have @ORGANIZATION1's @CAPS1 develop cases where their emotions do not show physically. This case is similar to the physical features of @CAPS3 @CAPS1. My main case with having @ORGANIZATION1's @CAPS1 is that I do not receive all the information given to me at one time. When people tell me a joke that they think is hilarious, I tend to miss the important parts of the joke, which are funny. As the joke comes to an end, everybody around me is laughing, but I am standing there with a blank look on my face. Sometimes I tell people I cannot laugh. Though I say this, I know it is not true. I can laugh, it is just really hard for me to find the right time. Most times, I laugh at a situation that really is not funny and everybody else has a blank gaze on their face. This usually makes me feel embarrassed and I don't feel normal. A lot of people tell me I look expressionless and then try to perk me up, or smile. I don't feel the need to smile, even in happy times in my life. I have not laughed enough, but have been learning in how to make bad situations into great and happy ones. Learning that at any moment I can laugh is important. Even if there is no joke being told, I can laugh. Most people would probably look a me in a weird way, but I wouldn't care because I am the one that feels happiness inside. Laughing relieves stress, makes other people feel good, and allows people with mental disabilities to feel more comfortable in the community they live in. I have finally learned that even with @ORGANIZATION1's @CAPS1, I can laugh just as much as anyone.",
                             "Dear @ORGANIZATION1, @CAPS1 more and more people start to use computers, goes more and more into the dark aged. While computer @MONTH1 be helpful in some cases, they are also making people excercize less, most @CAPS1 many people are nature, and more people are becoming awkward. So are computer advancement recently worth society becoming less interactive? The answer is no. With more thing to do on the computer people arent getting enough exercize to keep up a healthy regimens. Kids would rather waste time on the computer than go out play. @NUM1 years ago, when most that many people had computers, you could see people outside playing and having fun with each other, But now, when you look outside, no one is on the local playground having fun and exercising, they are on the internet."]) #output format
#display the interface
gui.launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://20028.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<fastapi.applications.FastAPI at 0x7f88870ef050>,
 'http://127.0.0.1:7864/',
 'https://20028.gradio.app')